# 预处理：提取出单个站点内的所有因子

In [4]:
ids_shangban = [ '天宝大水港排涝站','中排渠涝站（天宝）',
              '甘棠溪慧民花园监测点',
        '康山溪金峰花园监测点', '芗城水利局站','中山桥水闸站', '北京路水闸站','九湖监测点','桂林排涝站','上坂']


input_file,root_dir,\
include_site,include_factor,\
seq_length_x,seq_length_y = './data/water/shangban/water_4H.csv',\
  './data/water/shangban/singSiteMulFac',[0,1,2,3,6,8],\
    ids_shangban,24,3

import numpy as np
import os
import pandas as pd

In [ ]:
cols = [i + 3 for i in include_factor]
cols = [1, 2] + cols

df = pd.read_csv(input_file, usecols=cols)
merge = None
for site in include_site:
    one = df.loc[df['站点名称'] == site]
    one.columns = ['time', 'site'] + [site + factors[i] for i in include_factor]
    one.drop(columns=['site'], axis=1, inplace=True)
    merge = one if merge is None else pd.merge(merge, one, on='time')

merge.set_index(keys='time', inplace=True)

exi = os.path.exists(root_dir)
if not exi:
    os.mkdir(root_dir)
# 保存
merge.to_csv(root_dir+'/data.csv', encoding='utf_8_sig')

x_offsets = np.sort(np.concatenate((np.arange(-(seq_length_x - 1), 1, 1),)))
y_offsets = np.sort(np.arange(1, (seq_length_y + 1), 1))
x, y = generate_graph_seq2seq_io_data(
    merge,
    x_offsets=x_offsets,
    y_offsets=y_offsets,
    add_time_in_day=True,
)
print("x shape: ", x.shape, ", y shape: ", y.shape)
# 随机打乱！
per = np.random.permutation(x.shape[0])
x = x[per]
y = y[per]

# Write the data into npz file.
num_samples = x.shape[0]
num_test = round(num_samples * 0.2)
num_train = round(num_samples * 0.7)
num_val = num_samples - num_test - num_train
x_train, y_train = x[:num_train], y[:num_train]
x_val, y_val = (
    x[num_train: num_train + num_val],
    y[num_train: num_train + num_val],
)
x_test, y_test = x[-num_test:], y[-num_test:]

for cat in ["train", "val", "test"]:
    _x, _y = locals()["x_" + cat], locals()["y_" + cat]
    print(cat, "x: ", _x.shape, "y:", _y.shape)
    np.savez_compressed(
        os.path.join(root_dir+'/', f"{cat}.npz"),
        x=_x,
        y=_y,
        x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
        y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
    )


In [10]:
import torch
x = torch.tensor([1.0,2,3],requires_grad=True)
x_matrix = torch.zeros([4,4])
for i in range(3):
    x_matrix[i][i+1] = x[i]
adj = torch.zeros([4,4],requires_grad=True)
adj1 = adj + x_matrix
adj2 = adj1 * adj1
adj2 = adj2.sum()

# print(type(adj))
print(adj2)
adj2.backward()
# out = adj


print(x.grad)
print(adj.grad)

tensor(14., grad_fn=<SumBackward0>)
tensor([2., 4., 6.])
tensor([[0., 2., 0., 0.],
        [0., 0., 4., 0.],
        [0., 0., 0., 6.],
        [0., 0., 0., 0.]])


In [12]:
x_ma = torch.zeros([2,4,4])
print(x_ma)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
